In [18]:
import requests as r
import json

In [19]:
api_key = 'DEMO_KEY'

In [39]:
def get_product_list(q = ''):
#   https://api.nal.usda.gov/ndb/search/?format=json&q=butter&sort=n&max=25&offset=0&api_key=DEMO_KEY
    url_root = 'https://api.nal.usda.gov/ndb/search/?q={query}&format={format}&sort={sort}&max={records_limit}&&offset={offset}&api_key={api_key}'
    params = { 'format': 'json', 'offset': 0, 'records_limit': 10, 'sort': 'n', 'api_key': api_key, 'query': q }
    resp = r.get(url_root.format_map(params))
    data = json.loads(resp._content)
    result = list([(x['ndbno'], x['name']) for x in data['list']['item']])
    return result
    

In [42]:
get_product_list('carrot')

req
[('45167404', "ABE'S, COCONUT CARROT CAKE, UPC: 044261720871"), ('45187081', "ABE'S, MINI-MUFFINS, COCONUT CARROT CAKE, UPC: 044261490866"), ('45051561', 'A BLEND OF BROCCOLI, CARROTS, CAULIFLOWER, ITALIAN BEANS, ZUCCHINI & LIMA BEANS, UPC: 041497033560'), ('45045631', 'AHOLD, MATCHSTICK CARROTS, UPC: 688267149009'), ('45045780', "AHOLD, NATURE'S PROMISE, BABY FOOD, CARROTS, UPC: 688267156588"), ('45045793', "AHOLD, NATURE'S PROMISE, ORGANIC BABY FOOD, CARROTS PEAS, UPC: 688267156663"), ('45015211', "ANTONINA'S ARTISAN BAKERY, GLUTEN FREE CARROT CUPCAKE TOPPED WITH A CREAM CHEESE FROSTING, UPC: 898223002921"), ('45015213', "ANTONINA'S ARTISAN BAKERY, MINI PANCAKES, CARROT, UPC: 898223002914"), ('45186019', 'APRICOT GINGER GLAZED CARROTS, UPC: 815712012957'), ('45112705', "ARNIE'S, CARROT CAKE, UPC: 076185004911")]


In [84]:
def get_product_detail(product_id):
#   https://api.nal.usda.gov/ndb/reports/?ndbno=01009&type=f&format=json&api_key=DEMO_KEY
    url_root = 'https://api.nal.usda.gov/ndb/reports/?ndbno={product_id}&format={format}&type={type}&api_key={api_key}'
    params = { 'format': 'json', 'type': 'f', 'api_key': api_key, 'product_id': product_id }
    resp = r.get(url_root.format_map(params))
    data = json.loads(resp._content)
    item = data['report']['food']
    result = {}
    result['id'] = item['ndbno']
    result['name'] = item['name']
    nutrients = item['nutrients']
    for n_item in nutrients:
        if n_item['nutrient_id'] == '208':
           result['energy'] = (int(n_item['value']), n_item['unit'])
        if n_item['nutrient_id'] == '203':
           result['protein'] = (float(n_item['value']), n_item['unit'])
        if n_item['nutrient_id'] == '204':
           result['fat'] = (float(n_item['value']), n_item['unit'])
        if n_item['nutrient_id'] == '205':
           result['carbohydrate'] = (float(n_item['value']), n_item['unit'])
        if n_item['nutrient_id'] == '291':
           result['fiber'] = (float(n_item['value']), n_item['unit'])
        if n_item['nutrient_id'] == '269':
           result['sugar'] = (float(n_item['value']), n_item['unit'])
    return result

In [85]:
get_product_detail('45051561')

{'id': '45051561', 'name': 'A BLEND OF BROCCOLI, CARROTS, CAULIFLOWER, ITALIAN BEANS, ZUCCHINI & LIMA BEANS, UPC: 041497033560', 'energy': (40, 'kcal'), 'protein': (1.33, 'g'), 'fat': (0.0, 'g'), 'carbohydrate': (8.0, 'g'), 'fiber': (2.7, 'g'), 'sugar': (2.67, 'g')}
